In the basic tutorial we covered how to add static features, predictors and outcomes.
In this tutorial, we'll expand on that, covering how to effectively add many features by:
1. Utilising data loaders in a data_loaders registry,
2. Populating a dictionary from a long format dataframe,
3. Creating feature combinations from specifcations,
4. Using caching, so you can iterate on your datasets without having to complete full computations every time


# Using data loaders
Until now, we've loaded our data first and then created combinations. But what if your data lies in an SQL database, and you don't want to save it to disk?

Time to introduce feature loaders. All feature spec objects can resolve from a loader function. The only requirement of that loader function is that it should return a values dataframe, which should contain an ID column, a timestamp column and a list of vlaues. This means you can have loaders that load from REDIS, SQL databases, or just from disk. Whatever you prefer.

This function is then called when you initialise a specification.

This loader is specified in the values_loader key like so:


In [1]:
from skimpy import skim
from timeseriesflattener.testing.load_synth_data import load_synth_predictor_float
from timeseriesflattener.resolve_multiple_functions import mean
from timeseriesflattener.feature_spec_objects import PredictorSpec
from pprint import pprint
import numpy as np

In [2]:
pred_spec_batch = PredictorSpec(
    values_loader=load_synth_predictor_float,
    lookbehind_days=730,
    fallback=np.nan,
    resolve_multiple_fn=mean,
    feature_name="predictor_name",
)

pred_spec_batch.values_df

,dw_ek_borger,timestamp,value
0,9476,1969-03-05 08:08:00,0.816995
1,4631,1967-04-10 22:48:00,4.818074
2,3890,1969-12-15 14:07:00,2.503789
3,1098,1965-11-19 03:53:00,3.515041
4,1626,1966-05-03 14:07:00,4.353115
...,...,...,...
99995,4542,1968-06-01 17:09:00,9.616722
99996,4839,1966-11-24 01:13:00,0.235124
99997,8168,1969-07-30 01:45:00,0.929738
99998,9328,1965-12-22 10:53:00,5.124424


## The data loaders registry
If you inspect the source code of load_synth_predictor_float, you'll see that it is decorated with @data_loaders.register("synth_predictor_float").

```python
@data_loaders.register("synth_predictor_float")
def load_synth_predictor_float(
    n_rows: Optional[int] = None,
) -> pd.DataFrame:
    """Load synth predictor data.".

    Args:
        n_rows: Number of rows to return. Defaults to None which returns entire coercion data view.

    Returns:
        pd.DataFrame
    """
    return load_raw_test_csv("synth_raw_float_1.csv", n_rows=n_rows)
```

This registers it in the data_loaders registry under the "synth_predictor_float" key.

When you initialise a feature specification, it will look at the type of its `values_loader` attribute. If its type is a string, it will look for that string as a key in the data loaders registry. If it finds it, it'll resolve it to the value, in this case the `load_synth_predictor_float` function, and call that function.

The same concept applies for the resolve multiple functions.
This is super handy if you e.g. want to parse a config file, and therefore prefer to specify your data loaders as strings.

# Creating feature combinations
Manually specifying a handful of features one at a time is rather straightforward, but what if you want to generate hundreds of features? Or want to have multiple different lookbehind windows, e.g. a month, 6 months and a year? Then the amount of code you'll have to write will grow quite substantially and becomes time consuming and hard to navigate.

To solve this problem, we implemented feature group specifications. They allow you to combinatorially create features. Let's look at an example:


In [3]:
from timeseriesflattener.feature_spec_objects import PredictorGroupSpec
from timeseriesflattener.resolve_multiple_functions import maximum

In [4]:
pred_spec_batch = PredictorGroupSpec(
    values_loader=["synth_predictor_float"],
    lookbehind_days=[365, 730],
    fallback=[np.nan],
    resolve_multiple_fn=[mean, maximum],
).create_combinations()

You'll note that:

1. All attributes are now required to be lists. This makes iteration easier when creating the combinations.
2. We require values_loaders to be strings that can be resolved from the registry. This string is also used when creating the column names - otherwise we wouldn't know what to call the columns.

Let's check that the results look good.

In [5]:
# Create a small summary to highlight the generated predictors
pred_spec_batch_summary = [
    {
        "feature_name": pred_spec.feature_name,
        "lookbehind_days": pred_spec.lookbehind_days,
        "resolve_multiple_fn": pred_spec.key_for_resolve_multiple,
    }
    for pred_spec in pred_spec_batch
]
print(
    f"––––––––– We created {len(pred_spec_batch)} combinations of predictors. ––––––––––"
)
pprint(pred_spec_batch_summary)

––––––––– We created 4 combinations of predictors. ––––––––––
[{'feature_name': 'synth_predictor_float',
  'lookbehind_days': 365,
  'resolve_multiple_fn': 'mean'},
 {'feature_name': 'synth_predictor_float',
  'lookbehind_days': 730,
  'resolve_multiple_fn': 'mean'},
 {'feature_name': 'synth_predictor_float',
  'lookbehind_days': 365,
  'resolve_multiple_fn': 'maximum'},
 {'feature_name': 'synth_predictor_float',
  'lookbehind_days': 730,
  'resolve_multiple_fn': 'maximum'}]


Now we know how to create a bunch of feature specifications quickly! But with more features comes more computation. Let's look at caching next, so we can iterate on our datasets more quickly.

# Caching

Timeseriesflattener ships with a class that allows for caching to disk. Let's look at an example of that:

In [6]:
from skimpy import skim
from timeseriesflattener.testing.load_synth_data import load_synth_prediction_times
from timeseriesflattener.feature_cache.cache_to_disk import DiskCache
from timeseriesflattener.flattened_dataset import TimeseriesFlattener
from pathlib import Path

In [7]:
ts_flattener = TimeseriesFlattener(
    prediction_times_df=load_synth_prediction_times(),
    id_col_name="dw_ek_borger",
    timestamp_col_name="timestamp",
    n_workers=4,
    cache=DiskCache(feature_cache_dir=Path(".tmp") / "feature_cache"),
    drop_pred_times_with_insufficient_look_distance=True,
)

All we need to specify is that we use the DiskCache class, and which directory to save the feature cache to.

The first time we create features, this will just save them to disk and won't make any difference to performance. But say we want to add two more features - then it'll load the features that it has already computed from disk, and then only compute the two new features.

Note that DiskCache is an instance of the abstract class FeatureCache. If you want to implement your own cache, for example using REDIS or SQL, all you'll need is to implement the 3 methods in that class. Now, let's compute a dataframe to check that everything works.

In [8]:
ts_flattener.add_spec(pred_spec_batch)

In [9]:
df = ts_flattener.get_df()

2022-12-08 14:14:12 [INFO] There were unprocessed specs, computing...
2022-12-08 14:14:12 [INFO] _drop_pred_time_if_insufficient_look_distance: Dropped 4038 (0.4%) rows
100%|██████████| 4/4 [00:01<00:00,  3.08it/s]
2022-12-08 14:14:13 [INFO] Starting concatenation. Will take some time on performant systems, e.g. 30s for 100 features. This is normal.
2022-12-08 14:14:13 [INFO] Concatenation took 0.018 seconds
2022-12-08 14:14:13 [INFO] Merging with original df


In [10]:
skim(df)
df

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 5962   │ │ float64     │ 4     │                                                          │
│ │ Number of columns │ 7      │ │ int64       │ 1     │                                                          │
│ └───────────────────┴────────┘ │ datetime64  │ 1     │                                                          │
│                                │ string      │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name               ┃ NA    ┃ NA %   ┃ mean   ┃ sd    ┃ p0        ┃ p25   ┃ p75   ┃ p100   ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━┩  │
│ │ dw_ek_borger              │     0 │      0 │   5000 │  2900 │         0 │  2500 │  7400 │  10000 │ █▇███▇  │  │
│ │ pred_synth_predictor      │   820 │     14 │    6.6 │   2.6 │   0.00039 │   4.8 │   8.8 │     10 │ ▁▂▃▄▆█  │  │
│ │ pred_synth_predictor      │   820 │     14 │      5 │   2.1 │   0.00039 │   3.5 │   6.4 │     10 │ ▂▄██▄▂  │  │
│ │ pred_synth_predictor      │   110 │    1.8 │    7.7 │   2.1 │     0.058 │   6.7 │   9.3 │     10 │  ▁▁▂▄█  │  │
│ │ pred_synth_predictor      │   110 │    1.8 │      5 │   1.7 │     0.058 │   3.9 │   6.1 │    9.9 │ ▁▃██▃▁  │  │
│ └───────────────────────────┴───────┴────────┴────────┴───────┴───────────┴───────┴───────┴────────┴─────────┘  │
│                                                    datetime                                                     │
│ ┏━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓  │
│ ┃ column_name      ┃ NA   ┃ NA %    ┃ first                      ┃ last                       ┃ frequency    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩  │
│ │ timestamp        │    0 │       0 │    1967-01-02 01:16:00     │    1969-12-31 21:42:00     │ None         │  │
│ └──────────────────┴──────┴─────────┴────────────────────────────┴────────────────────────────┴──────────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                           ┃ NA    ┃ NA %      ┃ words per row            ┃ total words         ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ prediction_time_uuid                  │     0 │         0 │                        1 │                6000 │  │
│ └───────────────────────────────────────┴───────┴───────────┴──────────────────────────┴─────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

,dw_ek_borger,timestamp,prediction_time_uuid,pred_synth_predictor_float_within_365_days_maximum_fallback_nan,pred_synth_predictor_float_within_365_days_mean_fallback_nan,pred_synth_predictor_float_within_730_days_maximum_fallback_nan,pred_synth_predictor_float_within_730_days_mean_fallback_nan
0,9903,1968-05-09 21:24:00,9903-1968-05-09-21-24-00,0.154981,0.154981,2.194319,0.990763
1,6447,1967-09-25 18:08:00,6447-1967-09-25-18-08-00,8.930256,5.396017,9.774050,5.582745
2,4927,1968-06-30 12:13:00,4927-1968-06-30-12-13-00,6.730694,4.957251,6.730694,4.957251
3,5475,1967-01-09 03:09:00,5475-1967-01-09-03-09-00,9.497229,6.081539,9.497229,5.999336
4,3157,1969-10-07 05:01:00,3157-1969-10-07-05-01-00,5.243176,5.068323,5.243176,5.068323
...,...,...,...,...,...,...,...
5957,4228,1967-02-26 05:45:00,4228-1967-02-26-05-45-00,6.844010,4.353579,6.844010,3.792014
5958,9745,1969-02-04 01:18:00,9745-1969-02-04-01-18-00,3.858509,3.858509,3.858509,2.394074
5959,3385,1967-07-17 19:18:00,3385-1967-07-17-19-18-00,9.370554,5.463267,9.370554,5.769484
5960,1421,1968-04-15 15:53:00,1421-1968-04-15-15-53-00,8.972364,8.972364,8.972364,7.732447
